In [42]:
from load_netmob_data import load_subway_shp,load_netmob_gdf,find_ids_within_epsilon,tackle_all_days,calculate_grid_size,get_grid,match_tile_ids_with_grid,resize_tensor
from os import listdir
import os 
import pandas as pd 

import torch
import pickle 
import os
import numpy as np 

In [43]:
data_folder_path = '../../../data/'
save_folder = f"{data_folder_path}NetMob_tensor/"
netmob_data_folder_path = f"{data_folder_path}NetMob/"
epsilon=1000  #epsilon : radius, in meter (1000m) 
# W,H = 2*(epsilon//100 + 1), 2*(epsilon//100 + 1)

if not os.path.exists(save_folder):
    os.makedirs(save_folder)
    
# Load subway gdf adn NetMob gdf
ref_subway = load_subway_shp(folder_path = data_folder_path)
Netmob_gdf,working_zones = load_netmob_gdf(folder_path = netmob_data_folder_path,
                             data_folder = '../Data/lyon_iris_shapefile/', 
                             geojson_path = 'Lyon.geojson',
                             zones_path = 'lyon.shp')
Netmob_gdf_dropped = Netmob_gdf.drop_duplicates(subset = ['tile_id'])  # Some Doubles are exis

# Get Cell-Id within epsilon : 
result,joined = find_ids_within_epsilon(Netmob_gdf_dropped,ref_subway,epsilon=epsilon) 
maxi_nb_tile =  result.apply(lambda row: len(row.tile_id),axis=1).max()
print(f"Maximum number of NetMob Cell associated to a subway station: {maxi_nb_tile}")

Maximum number of NetMob Cell associated to a subway station: 318


In [44]:
#joined.explore('COD_TRG')

In [45]:
apps = [app for app in listdir(netmob_data_folder_path) if ((app != 'Lyon.geojson') and (not app.startswith('.'))) ]   # Avoid hidden folder and Lyon.geojson
Tensors = []
# For each app
if False:
    for app in apps: 
        print('App: ',app)
        metadata = {result['COD_TRG'][station_ind] : {} for station_ind in range(len(result))}
        folder_days = [day for day in listdir(f'{netmob_data_folder_path}/{app}') if (not day.startswith('.')) ]
        Tensors_days,metadata = tackle_all_days(result,metadata,netmob_data_folder_path,app,maxi_nb_tile,folder_days)
        torch.save(Tensors_days,f"{save_folder}{app}.pt")
        pickle.dump(metadata,open(f"{save_folder}{app}_metadata.pkl",'wb'))

## Load Tensor example:

In [46]:
app = 'Apple_Video'

Apple_Video_meta = pickle.load(open(f"{save_folder}{app}_metadata.pkl","rb"))
Apple_Video = torch.load(f"{save_folder}{app}.pt")  #[day, transfer_mode, Station, Tile_id, (hour,minutes)]
print(f"{app} Tensor: {Apple_Video.size()}")

Apple_Video Tensor: torch.Size([77, 2, 43, 318, 96])


In [47]:
# Input adapted to station i:
T = Apple_Video  #[Day, DL/UL, Nstation, Tile-id, (hour,minute)] 
metadata = Apple_Video_meta

stations = list(metadata.keys())

i = 0
station = stations[i]
tile_ids = np.array(metadata[station]['tile_id'])
T_i = torch.squeeze(T[:,:,i,:,:])  #[Day, DL/UL, Tile-id, (hour,minute)] 
# BIEN ETRE SUR DE CE PERMUTE RESHAPE §§§§
T_i = T_i.permute(1,2,0,3)  #[DL/UL, Tile-id,Day,(hour,minute)] 
T_i = T_i.reshape(T_i.size(0),T_i.size(1),-1) #[DL/UL, Tile-id, Day*(hour,minute)]
#  ................................
T_i = T_i.permute(2,0,1) #[Day*(hour,minute), DL/UL, Tile-id]   <-> [N,C,H*W]
print(f"Ti: {T_i.size()}")

Ti: torch.Size([7392, 2, 318])


In [49]:
step_south_north = 287

# Find dimensions
H, W, tile_ids_real = calculate_grid_size(tile_ids,epsilon)

# Get Grid 
grid = get_grid(tile_ids_real,H,W,step_south_north)

# Match Metadata with Grid 
positions = match_tile_ids_with_grid(tile_ids_real,grid.reshape(-1))

# Re-organize Tensor  
new_T_i = resize_tensor(T_i, H,W, positions)